In [1]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from data import BodyPart 
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.optimizers import Adam

tfjs_model_dir = 'model'

In [2]:
# loading final csv file
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    
    return X, y, classes

In [3]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center

In [4]:
def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.

    It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size

In [5]:
def normalize_pose_landmarks(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks

In [6]:
def landmarks_to_embedding(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    # Flatten the normalized landmark coordinates into a vector
    embedding = keras.layers.Flatten()(landmarks)
    return embedding

In [7]:
def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)

In [8]:
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')

In [9]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)
model.summary()


model.compile(
    optimizer = Adam(learning_rate=0.005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

2022-05-16 14:49:57.445112: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 34)]              0         
                                                                 
 dense (Dense)               (None, 128)               4480      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 13)                845       
                                                                 
Total params: 13,581
Trainable params: 13,581
Non-trainable p

In [21]:
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

In [12]:
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_data=(processed_X_val, y_val))
                    # callbacks=[checkpoint, earlystopping])

--------------TRAINING----------------
Epoch 1/200
12/12 [==============================] - 0s 6ms/step - loss: 0.1179 - accuracy: 0.9605 - val_loss: 0.3718 - val_accuracy: 0.9365
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2087 - accuracy: 0.9379 - val_loss: 0.4480 - val_accuracy: 0.9048
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2067 - accuracy: 0.9435 - val_loss: 0.4021 - val_accuracy: 0.9365
Epoch 4/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1924 - accuracy: 0.9492 - val_loss: 0.3890 - val_accuracy: 0.9365
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2025 - accuracy: 0.9435 - val_loss: 0.3756 - val_accuracy: 0.9365
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2034 - accuracy: 0.9435 - val_loss: 0.4614 - val_accuracy: 0.9365
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1877 - accuracy: 0.9407 - val_loss: 0.43

In [13]:
print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)

-----------------EVAUATION----------------
4/4 [==============================] - 0s 1ms/step - loss: 0.2642 - accuracy: 0.9391
LOSS:  0.2642013132572174
ACCURACY:  0.939130425453186


In [14]:
tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)

tfjs model saved at  model


In [15]:
model.predict(preprocess_data(X_val))

array([[3.27806867e-14, 8.17371388e-19, 1.15327104e-20, 1.14494332e-26,
        2.90138833e-15, 5.34582274e-11, 6.23879878e-30, 4.64316764e-22,
        9.67718700e-12, 1.00000000e+00, 3.33662499e-15, 1.36327409e-18,
        2.78417400e-10],
       [1.98561447e-17, 4.71510721e-21, 1.02489696e-12, 1.93648408e-16,
        3.50802831e-18, 8.15638200e-17, 1.52260432e-16, 4.25918659e-16,
        4.93279124e-14, 4.13382653e-12, 2.89451991e-12, 9.86779809e-01,
        1.32201836e-02],
       [4.92870896e-16, 9.99992490e-01, 7.84107293e-23, 8.04036164e-20,
        9.55016178e-29, 7.53417362e-06, 1.07911270e-27, 2.76015647e-34,
        0.00000000e+00, 0.00000000e+00, 1.03413714e-34, 4.08666029e-34,
        6.80754206e-35],
       [1.13576373e-35, 2.77265872e-21, 6.28759045e-10, 1.86452652e-18,
        2.45266531e-29, 3.11440282e-27, 4.40677361e-15, 4.03289176e-26,
        1.63954083e-13, 2.73967555e-22, 1.00000000e+00, 3.00979814e-12,
        6.81975960e-12],
       [2.77094229e-16, 2.32178094e-

In [16]:
print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_val, y_val)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)

-----------------EVAUATION----------------
2/2 [==============================] - 0s 2ms/step - loss: 0.2591 - accuracy: 0.9365
LOSS:  0.2590975761413574
ACCURACY:  0.9365079402923584
